## **Preparing pipeline input to Simulate pipeline job**

In [78]:
from google.cloud import storage
import time
import random
import json

In [79]:
"""Replace Bucket_name to the bucket name used in the project"""
bucket_name = f"{$Bucket_name$}"
storage_client = storage.Client()
bucket = storage_client.bucket(f'{bucket_name}')

In [80]:
def create_prompt_json(prompt_json,file_name):
    """
        Method to create prompt json / Prompts are given only to randomly picked images
        
        Parameters
        ----------
        prompt_json : Dict
            Prompts json with co-ordinate prompts
        file_name: str
            Image file name

        Returns
        -------
        prompt_json : Dict
            Updated Prompt Json
    """
    if random.randint(1,1000)%2 == 0:
        prompt_json[file_name] = {}
        prompt_json[file_name]["1"]= [[300, 200]]
    
    return prompt_json

In [81]:
"""Set the batch size to whatever you want to test the pipeline with"""
batch_sz = 5
batch_folder_name = f"batch_{batch_sz}"

##### Create a folder with name **"batch"** and upload **"simple_pencil.jpeg"** and **"simple_apple.jpeg"** files present in **"images"** folder inside **"demo-notebooks"**

##### The overall intent of doing this is to just simulate the pipeline job duplicating images with unique names and creating a batch of the size that we want

In [82]:
prompt_json = {}
""" batch is a folder inside pipeline folder with "simple_pencil.jpeg" and "simple_apple.jpeg" images"""
for i in range(batch_sz):
    if (i%2)==0:
        bucket.copy_blob(bucket.blob('pipeline/batch/simple_pencil.jpeg'), bucket, new_name=f'{batch_folder_name}/simple_pencil_{i}.jpeg')
        prompt_json = create_prompt_json(prompt_json, f"simple_pencil_{i}.jpeg")
    else:
        bucket.copy_blob(bucket.blob('pipeline/batch/simple_apple.jpeg'), bucket, new_name=f'{batch_folder_name}/simple_apple_{i}.jpeg')
        prompt_json = create_prompt_json(prompt_json, f"simple_apple_{i}.jpeg")

In [83]:
len(prompt_json)

4

In [84]:
""" Writing the prompts json to file"""
PROMPTS_JSON = f"json_prompts/prompts_json_{batch_sz}.jsonl"
with open(PROMPTS_JSON, "w") as outfile:
    json.dump(prompt_json, outfile)

In [85]:
"""Uploading the prompts JSON file to Cloud Storage"""
blob = bucket.blob(f"pipeline/batch_{batch_sz}/{PROMPTS_JSON}")

blob.upload_from_filename(PROMPTS_JSON)

In [86]:
prompt_json

{'simple_pencil_0.jpeg': {'1': [[300, 200]]},
 'simple_pencil_2.jpeg': {'1': [[300, 200]]},
 'simple_apple_3.jpeg': {'1': [[300, 200]]},
 'simple_pencil_4.jpeg': {'1': [[300, 200]]}}